In [1]:
import pandas as pd
from datetime import date
from math import log

filename = 'new_member_info.csv'
data = pd.read_csv(filename)
data[['FollowingItemsNum','FollowersNum','HelpfulMarksReceived','ForumPost_num']]*=1000
data = data.round(2)
data

,Name,Age,Sex,MainCondition,ActiveDays,ConditionsNum,FollowingItemsNum,FollowersNum,HelpfulMarksReceived,ForumPost_num,Target
0,0,0.30,M,amyotrophic lateral sclerosis,1093,1,10.97,10.05,8.23,2.74,3
1,1,1.06,M,amyotrophic lateral sclerosis,888,26,4.50,19.12,2.25,3.37,3
2,2,-0.08,F,amyotrophic lateral sclerosis,1028,1,135.08,29.15,0.97,0.97,3
3,3,0.00,M,amyotrophic lateral sclerosi,929,1,2.15,1.08,2.15,5.38,3
4,4,-0.77,M,amyotrophic lateral sclerosis,1502,1,31.94,29.94,49.90,43.91,3
...,...,...,...,...,...,...,...,...,...,...,...
145,145,0.30,M,Parkinson's diseas,20,1,47.62,47.62,47.62,47.62,0
146,146,0.43,F,Parkinson's disease,2525,1,0.40,0.40,0.40,0.40,3
147,147,1.57,M,Parkinson's diseas,174,1,5.71,5.71,5.71,5.71,0
148,148,0.49,M,Parkinson's disease,270,1,3.69,3.69,3.69,3.69,1


In [2]:
def getBestInformationGain(data):
    best = (0, 0,0)
    for label in ['Age','FollowingItemsNum','FollowersNum','HelpfulMarksReceived','ForumPost_num']:
        criterias = sorted(list(set(data[label])))
        E = -sum([((j/len(data))*log((j/len(data)),2)) for j in data['Target'].value_counts()])
        
        for i in range(len(criterias)-1):
            criteria = round(((criterias[i]+criterias[i+1])/2),3)
            A = data[data[label]<criteria]
            B = data[data[label]>criteria]
            EA = -sum([((j/len(A))*log((j/len(A)),2)) for j in A['Target'].value_counts()])
            EB = -sum([((j/len(B))*log((j/len(B)),2)) for j in B['Target'].value_counts()])
            IG = E - ((len(A)/len(data))*EA) - ((len(B)/len(data))*EB)
            if best[2] <IG:
                best = (label, criteria, IG)
                best_A = A[:]
                best_B = B[:]
    return best, best_A, best_B

In [3]:
class Node(object):
    def __init__(self, data, index = 1,depth = 0, label=None, criteria=None):
        self.data = data
        self.criteria = criteria
        self.index = index
        self.depth = depth
        if len(data['Target'].value_counts()) == 0:
            self.left = self.right = self.IG = self.target = None
            print("Error : Num of node is Zero..")
        elif len(data['Target'].value_counts()) == 1:
            self.left = self.right = self.IG = None
            self.target = (data['Target'].value_counts()).index[0]
        else:
            (label, criteria, IG),A,B = getBestInformationGain(data)
            self.left = Node(A,index*2,depth+1,label,'less than ' + str(criteria))
            self.right = Node(B,index*2+1,depth+1,label,'more than ' + str(criteria))
            self.target = None
        self.label = label
        
    
    def printNode(self,indent='',r = False):
        if self.target!=None:
            if r:
                print(indent + 'ㄴ'+'-'*3+('%14s' % self.criteria)+' / ')
                print(indent + ' '*5+'%%class%% : ',self.target)
            else:
                print(indent+'|'+'-'*3+('%14s' % self.criteria)+' / ')
                print(indent + '|'+' '*4+'%%class%% : ',self.target)
        else:
            if self.criteria == None:
                print(indent + self.label)
            elif r:
                print(indent + 'ㄴ'+'-'*3+('%14s' % self.criteria)+' / ')
                print(indent + ' '*5+self.label)
                indent = indent+'      '
            else:
                print(indent+'|'+'-'*3+('%14s' % self.criteria)+' / ')
                print(indent + '|'+' '*4+self.label)
                indent = indent+'|     '
            if self.left!=None:
                self.left.printNode(indent)
            if self.right!=None:
                self.right.printNode(indent,True)

tree = Node(data)
tree.printNode()

HelpfulMarksReceived
|---less than 3.02 / 
|    HelpfulMarksReceived
|     |---less than 1.36 / 
|     |    %%class%% :  3
|     ㄴ---more than 1.36 / 
|          HelpfulMarksReceived
|           |---less than 2.13 / 
|           |    FollowingItemsNum
|           |     |---less than 0.865 / 
|           |     |    %%class%% :  3
|           |     ㄴ---more than 0.865 / 
|           |          %%class%% :  2
|           ㄴ---more than 2.13 / 
|                FollowingItemsNum
|                 |---less than 2.905 / 
|                 |    FollowingItemsNum
|                 |     |---less than 1.445 / 
|                 |     |    %%class%% :  3
|                 |     ㄴ---more than 1.445 / 
|                 |          FollowersNum
|                 |           |---less than 1.27 / 
|                 |           |    %%class%% :  3
|                 |           ㄴ---more than 1.27 / 
|                 |                %%class%% :  2
|                 ㄴ---more than 2.905 / 
|             

In [ ]:
    
"""
    def find(self, key):
        return self._find_value(self.root, key)

    def _find_value(self, root, key):
        if root is None or root.data == key:
            return root is not None
        elif key < root.data:
            return self._find_value(root.left, key)
        else:
            return self._find_value(root.right, key)
"""